<a href="https://colab.research.google.com/github/SaifAlmaliki/chatbot-nltk/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import nltk
from nltk.stem import  WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

**Import and load the data file**

The data file is in JSON format (intents.json) so we used the json package to parse the JSON file into Python.

In [11]:
words = []
documents = []
classes = []
ignore_words = ['?', '!']

# Load json file
data_file = open('data/data.json', encoding='utf8').read()
intents = json.loads(data_file)

In [12]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

**Preprocess data**

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. 
Tokenizing is the most basic and first thing you can do on text data. 

Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [13]:
# iterate through the patterns and tokenize the sentence using nltk.word_tokenize()
nltk.download('punkt')

for intents in intents['intents']:
    for patterns in intents['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(patterns)
        words.extend(w)

        # add documents in the corpus
        documents.append((w, intents['tag']))

        #create a list of classes for our tags.
        # add to our classes list
        if intents['tag'] not in classes:
            classes.append(intents['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BRAVO15\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now we will lemmatize each word and remove duplicate words from the list. 

Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [14]:
nltk.download('wordnet')

#  lemmatize each word and remove duplicate words from the list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# sort words
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), 'documents')

# classes = intents
print(len(classes), 'classes', classes)

# words = all words, vocabulary
print(len(words), 'uniqe lemmatized words', words)

# creating a pickle file to store the Python objects which we will use while predicting.
pickle.dump(words, open('data/words_vn_en.pkl', 'wb'))
pickle.dump(classes, open('data/classes_vn_en.pkl', 'wb'))

115 documents
27 classes ['Buổi sáng cùng Tony', 'Danh mục sách', 'Hướng dẫn thanh toán', 'Mua sách', 'Mắt Biếc', 'Nguyễn Nhật Ánh', 'Nhà giả kim', 'Phản hồi sản phẩm', 'Thể loại tiểu thuyết', 'Tác giả', 'Tìm kiếm sách', 'book_search', 'cảm ơn', 'goodbye', 'greeting', 'lời chào', 'lựa chọn', 'options', 'search_book_by_name', 'sách gì bán chạy nhất', 'thanks', 'tại sao bán chạy', 'tạm biệt', 'whatIsAnBestselleredbook', 'whatthis bookAreBestsellered', 'whyBestsellered', 'địa chỉ']
208 uniqe lemmatized words ["'s", ',', '.', '16', 'a', 'abc', 'ai', 'am', 'and', 'anthony', 'anyone', 'are', 'awesome', 'be', 'become', 'bee', 'best', 'best-selling', 'bestsellered', 'big', 'biếc', 'biệt', 'book', 'buổi', 'bye', 'bán', 'bạn', 'bằng', 'bị', 'can', 'chatting', 'cho', 'chuyện', 'chào', 'chúc', 'chạy', 'chỉ', 'chỗ', 'could', 'cuốn', 'các', 'cách', 'có', 'cùng', 'cảm', 'của', 'cứu', 'cửa', 'data', 'day', 'detail', 'diary', 'do', 'doe', 'dẫn', 'fauci', 'find', 'for', 'giao', 'giúp', 'giả', 'go', 'gon

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BRAVO15\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Create training and testing data**

Now, we will create the training data in which we will provide the input and the output. 

Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [15]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0) 
    
    # output is '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[: , 0])
train_y = list(training[:, 1])

print("Training data created")
print("training data shape", training.shape)

Training data created
training data shape (115, 2)


C:\Users\BRAVO15\AppData\Local\Temp/ipykernel_5504/22447722.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


**Build the model**

We have our training data ready, now we will build a deep neural network that has **3 layers.** 

We use the Keras sequential API for this. After training the model for **200 epochs**, we achieved 100% accuracy on our model. 

Let us save the model as ‘chatbot_model.h5’.

In [16]:
# create 3 layers. First layer 128 neurons,
# second layer 64 neurons
# 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax


model = Sequential()
model.add(Dense(128, 
                input_shape=(len(train_x[0]),),
                activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Compile model. Stochastic gradient descent with Nesterov accelerated gradient 
# gives good results for this model
sgd = SGD(lr = 0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), 
                 np.array(train_y), 
                 epochs=200, 
                 batch_size=5, 
                 verbose=1)
model.save('ChatbotModel_vn_en.h5', hist)
print('Model created')

C:\Users\BRAVO15\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
23/23 [==============================] - 0s 727us/step - loss: 3.2921 - accuracy: 0.0783
Epoch 2/200
23/23 [==============================] - 0s 727us/step - loss: 3.1445 - accuracy: 0.1565
Epoch 3/200
23/23 [==============================] - 0s 727us/step - loss: 2.9659 - accuracy: 0.1913
Epoch 4/200
23/23 [==============================] - 0s 773us/step - loss: 2.8145 - accuracy: 0.2870
Epoch 5/200
23/23 [==============================] - 0s 773us/step - loss: 2.6043 - accuracy: 0.2783
Epoch 6/200
23/23 [==============================] - 0s 955us/step - loss: 2.5457 - accuracy: 0.3130
Epoch 7/200
23/23 [==============================] - 0s 1ms/step - loss: 2.2659 - accuracy: 0.3565
Epoch 8/200
23/23 [==============================] - 0s 1ms/step - loss: 2.0438 - accuracy: 0.4696
Epoch 9/200
23/23 [==============================] - 0s 1ms/step - loss: 1.8136 - accuracy: 0.5304
Epoch 10/200
23/23 [==============================] - 0s 1ms/step - loss: 1.7396 - accuracy: 0.45